In [ ]:
# Библиотеки для синтеза данных
# import datetime # Для подсчета времени и работы с файловой системой
import numpy as np # Для работы с матрицами 
import sys # Для специльного вывода
import matplotlib.pyplot as plt # Для виузализации 

from google.colab import drive
from google.colab import files # Импортируем Модуль files для работы с файлами
import time
import math
import random # Для генерации случайных чисел
import os

from PIL import Image, ImageOps

# Библиотеки для обучения модели
from tensorflow.keras.models import Model # Импортируем модели keras: Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, concatenate, Activation, MaxPooling2D, Conv2D, BatchNormalization, Flatten, Dense, Dropout, Conv2DTranspose, Concatenate, Reshape
from tensorflow.keras import backend as K # Импортируем модуль backend keras'а
from tensorflow.keras.optimizers import Adam # Импортируем оптимизатор Adam
from tensorflow.keras import utils # Импортируем модуль utils библиотеки tensorflow.keras для получения OHE-представления
from tensorflow.keras.preprocessing import image # Библиотека от Keras для загрузки изображений
from sklearn.model_selection import train_test_split

In [ ]:
# Монтируем наш Google-диск
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Удаление директории с распакованными данными (если она есть)
!rm -rf /content/people-dogs

In [ ]:
!unzip -q '/content/drive/MyDrive/Colab Notebooks/Machine_learning/Datasets/people-dogs.zip' -d /content/
# УДАЛИТЬ ФАЙЛ desktop.ini, который сам появляется!
db     = '/content/people-dogs'

# Перевод в вероятности (One Hot Encoding)
labels = {} # словарь
labels['dogs']  = [1,0]
labels['people'] = [0,1]

In [ ]:
# Функция умной обрезки
def smart_trimming(img, t='dogs'):
  img_w, img_h = img.size   # Получение размера картинки
  target_size = [120, 120]  # Размер, к которому будут приведены все изображения для н.с.

  # Проверка на формат
  if img_w > img_h:
    new_h = target_size[1]                # Высота - основная ось
    new_w = round(new_h / img_h * img_w)  # Вычисление ширины
  else:
    new_w = target_size[0]                # Ширина - основная ось
    new_h = round(new_w / img_w * img_h)  # Вычисление высоты

  # Применение измениний касательно размеров
  img = img.resize((new_w, new_h), Image.ANTIALIAS)
  # Вычисление центра картинки
  center = [new_w//2, new_h//2]

  # Установка параметров обрезки изображений в зависимости от того, что на них изображено
  if t == 'dogs':
    # Нахождение левой верхней и правой нижней точки для прямоугольной обрезки
    top_left = [center[0] - target_size[0]//2, center[1] - target_size[1]//2]
    bottom_right = [center[0] + target_size[0]//2, center[1] + target_size[1]//2]
  else:
    top_left = [center[0] - target_size[0]//2, 0]
    bottom_right = [center[0] + target_size[0]//2, target_size[1]]

  # Обрезка изображения
  img = img.crop((top_left[0], top_left[1], bottom_right[0], bottom_right[1]))

  return img

In [ ]:
# Трансформации для генерации данных

# Поворот изображения в градусах
rotation = [-9, -6, 0, 6, 9]

# Приближене изображени в процентах
zoom = [0, 6, 12]

# Размер финального изображения
final_size = [ 192, 108 ]

In [ ]:
# Проход по всем изображениям и генерация новых

dir_list = os.listdir(db)
total_files = 0
for dir in dir_list:
  cfile = 0 # номер текущего файла

  fid = os.listdir(db+"/"+dir)
  for f in fid:
    if '.jpg' not in f:
      continue

    cfile += 1
    total_files +=1
    print(cfile, "из", len(fid), "->",f)

    file_path = db+"/"+dir+"/"+f

    img_obj = Image.open(file_path)
    img_obj = img_obj.convert('L')

    width  = img_obj.size[0]    # Получаем ширину
    height = img_obj.size[1]    # Получаем высоту

    img_obj_crop = smart_trimming(img_obj, dir)

    width = 120
    height = 120

    # Вывод изображения и его обрезанной версии
    # plt.imshow(img_obj, cmap='gray')
    # plt.show()
    # plt.imshow(img_obj_crop, cmap='gray')
    # plt.show()


    # Трансформирование изображений
    img_obj_t = img_obj_crop.copy()
    for r in rotation:
      if r != 0:
        # Сохранение неповернутого изображения в качестве фона
        img_obj_tb = img_obj_t.copy()
        img_obj_tb = img_obj_tb.convert('RGBA') # Возращение цвета
        img_background_obj = img_obj_tb.copy()

        # Поворот изображения
        img_obj_tb = img_obj_tb.rotate(r)

        # Наложение на фон
        img_background_obj.paste(img_obj_tb, (0,0), img_obj_tb)

        # Конвертация обратно в изображение без альфа-канала
        img_obj_tr = img_background_obj.convert('RGB')
      else:
        img_obj_tr = img_obj_t.copy()

    for z in zoom:
      if z != 0:
        newsize = [ int(width + width/100*z) , int(height + height/100*z) ]
        img_obj_trz = img_obj_tr.resize(newsize)
      else:
        newsize = [ width, height ]
        img_obj_trz = img_obj_tr.copy()

      # Вычисление точек для обрезки
      p1 = (newsize[0] - width) // 2
      p2 = (newsize[1] - height) // 2
      p3 = p1 + width
      p4 = p2 + height

      # Обрезка
      img_obj_trzc = img_obj_trz.crop((p1, p2, p3, p4))

      # Формирование имени файла и сохранение в папку с исходными данными
      fname_obj = 'z' + str(z) + "_r" + str(r) + "_" + f
      fpath_obj = db + "/" + dir + "/" + fname_obj

      img_obj_trzc.convert("L") # Для чего эта конвертация?
      img_obj_trzc.save(fpath_obj, "JPEG") # Не смотря на указание JPEG, данные всё равно в jpg
print("ВСЕГО: ", total_files)

1 из 290 -> kolonin_gleb_2.jpg
2 из 290 -> kutin nikolai_02.jpg
3 из 290 -> marat_17.jpg
4 из 290 -> matrosova_18.jpg
5 из 290 -> Volynov_4.jpg
6 из 290 -> shmax_6.jpg
7 из 290 -> shmax_9.jpg
8 из 290 -> kutin nikolai_15.jpg
9 из 290 -> Grebyonkin13.jpg
10 из 290 -> kolonin_gleb_6.jpg
11 из 290 -> kutin nikolai_09.jpg
12 из 290 -> kutin nikolai_16.jpg
13 из 290 -> khachatryan_david_14.jpg
14 из 290 -> matrosova_02.jpg
15 из 290 -> shmax_3.jpg
16 из 290 -> khachatryan_david_07.jpg
17 из 290 -> CostasBG_8.jpg
18 из 290 -> alexandra_bashiyants_25.jpg
19 из 290 -> SokovEA.jpg
20 из 290 -> SokovASSA.jpg
21 из 290 -> kolonin_gleb_9.jpg
22 из 290 -> Asatryan_4.jpg
23 из 290 -> alexandra_bashiyants_27.jpg
24 из 290 -> matrosova_14.jpg
25 из 290 -> khachatryan_david_15.jpg
26 из 290 -> manojlovich_melanija_11.jpg
27 из 290 -> manojlovich_melanija_09.jpg
28 из 290 -> kuchmar19.jpg
29 из 290 -> Grebyonkin12.jpg
30 из 290 -> alexandra_bashiyants_32.jpg
31 из 290 -> khachatryan_david_01.jpg
32 из 2

In [ ]:
def load_image(file_name):
  image = Image.open(file_name) # Загружаем изображение из файла

  image = image.convert('L')  # Приводим изображение к режиму черно-белого изображения 8 бит (0 ... 256)

  image_width  = image.size[0]  # Получаем ширину
  image_height = image.size[1]  # Получаем высоту

  image_data = image.getdata()          # Получаем данные картинки
  image_data = np.array(image_data)     # Преобразуем в np массив

  image = image_data.reshape(image_height, image_width, 1)  # Решейпим изображение для цветного варианта

  # Предупреждение, что изображение не имеет стандартный размер
  if (image_height != 120) or (image_width != 120):
    print(file_name, "!!!!!!!!!!!!!!")

  return image # Возврат трехмерного массива пикселей

In [ ]:
# Формирование списка всех фалов

image_list = []

for label in labels.keys():
  flist = os.listdir(db + "/" + label)
  print (label, '->', len(flist), 'изображений')
  for image_name in flist:
    if ".jpg" not in image_name:
      continue
    image_path = db + "/" + label + "/" + image_name
    image_list.append(image_path)

dogs -> 1105 изображений
people -> 1151 изображений


In [ ]:
print()
print("Пример названий файлов в массиве:")
print(image_list[0:3])

# перемешаем значения в датасете
random.shuffle(image_list)

print()
print("Пример названий файлов в массиве после перемешивания:")
print(image_list[0:3])

print()


Пример названий файлов в массиве:
['/content/people-dogs/dogs/z12_r9_pestretsov_nikita_11.jpg', '/content/people-dogs/dogs/z6_r9_shmax_6.jpg', '/content/people-dogs/dogs/z6_r9_matrosova_19.jpg']

Пример названий файлов в массиве после перемешивания:
['/content/people-dogs/dogs/z12_r9_pestretsov_nikita_14.jpg', '/content/people-dogs/dogs/z12_r9_kutin nikolai_17.jpg', '/content/people-dogs/dogs/z6_r9_Asatryan_s10.jpg']



In [ ]:
# Формирование обучающей выборки

x_train = []
y_train = []
c = 1
total_files = len(image_list)
for image_name in image_list:
  img = load_image(image_name)
  label = image_name.split('/')[3]
  x_train.append(img)
  y_train.append(labels[label])
  print(c, total_files, image_name)
  c += 1

x_train = np.array(x_train)
y_train = np.array(y_train)

1 2252 /content/people-dogs/dogs/z12_r9_pestretsov_nikita_14.jpg
2 2252 /content/people-dogs/dogs/z12_r9_kutin nikolai_17.jpg
3 2252 /content/people-dogs/dogs/z6_r9_Asatryan_s10.jpg
4 2252 /content/people-dogs/dogs/z0_r9_kolonin_gleb_12.jpg
5 2252 /content/people-dogs/dogs/z6_r9_kolonin_gleb_17.jpg
6 2252 /content/people-dogs/people/z12_r9_kuchmar16.jpg
7 2252 /content/people-dogs/dogs/z12_r9_Grebyonkin-8.jpg
/content/people-dogs/dogs/alexandra_bashiyants_17.jpg !!!!!!!!!!!!!!
8 2252 /content/people-dogs/dogs/alexandra_bashiyants_17.jpg
/content/people-dogs/people/radchenko_012.jpg !!!!!!!!!!!!!!
9 2252 /content/people-dogs/people/radchenko_012.jpg
10 2252 /content/people-dogs/people/z0_r9_Asatryan_9.jpg
11 2252 /content/people-dogs/people/z6_r9_kutin nikolai_12.jpg
/content/people-dogs/dogs/marat_7.jpg !!!!!!!!!!!!!!
12 2252 /content/people-dogs/dogs/marat_7.jpg
13 2252 /content/people-dogs/dogs/z6_r9_marat_5.jpg
14 2252 /content/people-dogs/people/z12_r9_matrosova_03.jpg
15 2252 /con

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [ ]:
# Вывод размерности обучающей выборки
print("Размерность x_train", x_train.shape) # Имеет странную размерность. Может быть связано с форматом .jpg
print()
print("Размерность y_train", y_train.shape)

Размерность x_train (2252,)

Размерность y_train (2252, 2)


In [ ]:
# Разобъем обучающую выборку на выборки

# Проверочная выборка
x_val = x_train[2000:]
y_val = y_train[2000:]

# Обучающая выборка
x_train = x_train[0:2000]
y_train = y_train[0:2000]

print("Обучающая выборка:")
print(x_train.shape)
print(y_train.shape)
print()

print("Проверочная выборка:")
print(x_val.shape)
print(y_val.shape)
print()

Обучающая выборка:
(2000,)
(2000, 2)

Проверочная выборка:
(252,)
(252, 2)



In [ ]:
# задаём batch_size
batch_size = 32 

# Создаем последовательную модель
model = Sequential()
# Слой пакетной нормализации
model.add(BatchNormalization(input_shape=(120, 120, 1), name="bn1"))
# Первый сверточный слой
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', name="Conv2D-layer1"))
# Второй сверточный слой
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', name="Conv2D-layer2"))
# Первый слой уменьшения изображения в 2 раза
model.add(MaxPooling2D(pool_size=(2, 2), name="mp2D-layer1"))
# Слой Dropout
model.add(Dropout(0.25, name="Dropout-layer1"))

# Слой пакетной нормализации
model.add(BatchNormalization(name = "bn2"))
# Третий сверточный слой
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', name = "Conv2D-layer3"))
# Четвертый сверточный слой
model.add(Conv2D(64, (3, 3), padding='same', activation='relu', name = "Conv2D-layer4"))
# Второй слой уменьшения изображения в 2 раза
model.add(MaxPooling2D(pool_size=(2, 2), name="mp2D-layer2"))
# Слой Dropout
model.add(Dropout(0.25, name="Dropout-layer2"))

# Слой пакетной нормализации
model.add(BatchNormalization(name = "bn3"))
# Пятый сверточный слой
model.add(Conv2D(128, (3, 3), padding='same', activation='relu', name = "Conv2D-layer5"))
# Шестой сверточный слой
model.add(Conv2D(128, (3, 3), padding='same', activation='relu', name = "Conv2D-layer6"))
# Третий слой уменьшения изображения в 2 раза
model.add(MaxPooling2D(pool_size=(2, 2), name="mp2D-layer3"))
# Слой регуляризации Dropout
model.add(Dropout(0.25, name="Dropout-layer3"))

# Слой преобразования данных из 2D представления в плоское
model.add(Flatten(name="flatten"))
# Выходной полносвязный классифицирующий слой
model.add(Dense(2, activation='softmax', name = "labels"))

# Компилируем сеть
#model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.01), metrics=["accuracy"])
model.compile(loss="categorical_crossentropy", optimizer=Adam(), metrics=["accuracy"])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bn1 (BatchNormalization)    (None, 120, 120, 1)       4         
                                                                 
 Conv2D-layer1 (Conv2D)      (None, 120, 120, 32)      320       
                                                                 
 Conv2D-layer2 (Conv2D)      (None, 120, 120, 32)      9248      
                                                                 
 mp2D-layer1 (MaxPooling2D)  (None, 60, 60, 32)        0         
                                                                 
 Dropout-layer1 (Dropout)    (None, 60, 60, 32)        0         
                                                                 
 bn2 (BatchNormalization)    (None, 60, 60, 32)        128       
                                                                 
 Conv2D-layer3 (Conv2D)      (None, 60, 60, 64)        1

In [ ]:
# Обучаем сеть на наших данных
history = model.fit(x_train, 
                    y_train, 
                    batch_size=batch_size, 
                    epochs=5,
                    validation_data=(x_val, y_val),
                    verbose=1)

# Отображаем график точности обучения
plt.plot(history.history['accuracy'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_accuracy'], 
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

ValueError: ignored

In [ ]:
# Тест нейросети на своём изображении

uploaded = files.upload()

print(uploaded.keys())

# for fn in uploaded.keys():
#   print('Загружен файл "{name}" длиной {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

# print()

# for fn in uploaded.keys():
#   imgFile = fn
#   break

# img = Image.open(imgFile)
# img = img.convert('L')
# img = smart_trimming(img)

In [ ]:
# Отрисовка картинки
plt.imshow(img, cmap="gray")
plt.show()

image_width  = img.size[0]  # Вычисление ширины
image_height = img.size[1]  # Вычисление высоты

image_data = np.array(img.getdata())  # Получение данных картинки

image = image_data.reshape(image_height, image_width, 1)

predict = model.predict(np.array([image]))

print(predict)

labels_text = ['Собака', 'Человек']
n = 0
print("Распознан объект: ", np.argmax(predict[n]), "-", labels_text[np.argmax(predict[n])])